# Pre_unfolding

This notebook prepares input histograms for neutrino energy unfolding analysis in Solid Hydrogen using ROOT (PyROOT) and RDataFrame.

## Overview
It loads reconstructed event data from a ROOT file, applies selection cuts to isolate CC-QE signal events on hydrogen, and generates histograms needed for unfolding:
- **Response matrix**: true vs reconstructed neutrino energy for selected signal events
- **True energy distribution**: true neutrino energy spectrum for signal events
- **Reconstructed energy distributions**: separately for plastic (H) target and graphite (C) target
- **Acceptance and efficiency**: detector acceptance and reconstruction efficiency as functions of neutrino energy

Results are saved to `Plots/SH_pre_unfolding/SH_pre_unfolding.root` with all histograms needed for the unfolding procedure.

## Usage
Set the input ROOT file path and output directory in the first code cell, then run cells sequentially.

## Dependencies
- ROOT (with PyROOT support)
- RDataFrame
- Local helper module: `df_functions_utils.py` (provides dataframe filtering functions)


In [ ]:
import ROOT
from ROOT import TCanvas
from ROOT import TLegend
from ROOT import RDataFrame
from ROOT import kRed, kBlue
from ROOT import gStyle
import df_functions_utils as df_utils

df = RDataFrame("Rough_tree", "/storage/gpfs_data/neutrino/users/battisti/hydrogen_analysis_tests/Solid_Hydrogen2/Snap_file_processing/Processed_tree.root")
Out_dir = "Unfolding_output/"

In [ ]:
### Uncomment for fast testing
df = df.Range(1000)

### Generating output file

In [ ]:
f_out = ROOT.TFile(Out_dir + "SH_pre_unfolding.root", "RECREATE")

# Generating df with additional columns

### List of dataframes:
- df = contains Rough_tree with all its columns
- df_extend =  df + additional columns needed in the analysis 
- df_flag = df_extended + flags that identify the filters  

### Filters flags for df_flag:
- inside the STT volume (volume_filter)
- successufull reconstruction (reco_filter)
- one track (track_filter)
- neutron space correspondence (space_filter, done with clusters) (spaceTRUE_filter, done with hit segment starts)
- neutron time correspondence (time_filter, done with clusters) (timeTRUE_filter, done with hit segment starts)
- events CC-QE with antimuon and neutron in the final state, on H (true_filter) 

In [ ]:
df_extended = df_utils.df_extend(df)
df_flag = df_utils.df_addflags(df_extended)

# Plotting

HISTOGRAMS FOR THE UNFOLDING

response matrix

In [ ]:

h_matrix = df_flag.Filter('true_flag == 1 && volume_flag == 1 && reco_flag == 1 && track_flag == 1 && space_flag == 1 && time_flag == 1 && target == "tgt:1000010010"').Histo2D(("h_matrix", " response matrix, true selected CCQE on H ", 12, 0, 6000, 12, 0, 6000), "nu_Ereco", "Enu")
ROOT.gStyle.SetOptStat(0)

c_m = TCanvas("c", "Canvas", 600, 500)
c_m.SetLeftMargin(0.12)
c_m.SetRightMargin(0.15)   # serve spazio per la barra dei colori
c_m.SetBottomMargin(0.12)
c_m.SetTopMargin(0.08)
# c.SetLogz()
#c.SaveAs("nu_E_RecoVsTrue_25_09.png")
h_matrix.Draw("COLZ")

c_m.Draw()
c_m.SaveAs(Out_dir + "nu_E_Response_Matrix.pdf")

f_out.cd()
h_matrix.GetValue().Write()

True signal true energy 

In [ ]:
#sono i true anche selezionati o solo i true?
ROOT.gStyle.SetOptStat(0)

nbin = 12

h_true = df_flag.Filter('true_flag == 1 && volume_flag == 1 && target == "tgt:1000010010"').Histo1D(("h_true", "h_true", nbin, 0, 6000),"Enu")

c_t = TCanvas("c", "Canvas", 600, 500)
c_t.SetLeftMargin(0.12)
c_t.SetRightMargin(0.15)   # serve spazio per la barra dei colori
c_t.SetBottomMargin(0.12)
c_t.SetTopMargin(0.08)
h_true.Draw("COLZ")

c_t.Draw()
c_t.SaveAs(Out_dir + "nu_E_True_Distribution.pdf")

f_out.cd()
h_true.GetValue().Write()


True signal reco energy 

In [ ]:

ROOT.gStyle.SetOptStat(0)

nbin = 12

h_reco = df_flag.Filter('volume_flag == 1 && reco_flag == 1 && track_flag == 1 && space_flag == 1 && time_flag == 1 && target == "tgt:1000010010"').Histo1D(("h_reco", "h_reco", nbin, 0, 6000),"nu_Ereco")


c_r = TCanvas("c", "Canvas", 600, 500)
c_r.SetLeftMargin(0.12)
c_r.SetRightMargin(0.15)   # serve spazio per la barra dei colori
c_r.SetBottomMargin(0.12)
c_r.SetTopMargin(0.08)
h_reco.Draw("COLZ")

c_r.Draw()
c_r.SaveAs(Out_dir + "nu_E_Reco_Distribution.pdf")

f_out.cd()
h_reco.GetValue().Write()

selection on CH 

In [ ]:

ROOT.gStyle.SetOptStat(0)

nbin = 12

CH_reco = df_flag.Filter('volume_flag == 1 && reco_flag == 1 && track_flag == 1 && space_flag == 1 && time_flag == 1 && Where_int == "C3H6" ').Histo1D(("CH_reco", "CH_reco", nbin, 0, 6000),"nu_Ereco")


c_CH = TCanvas("c", "Canvas", 600, 500)
c_CH.SetLeftMargin(0.12)
c_CH.SetRightMargin(0.15)   # serve spazio per la barra dei colori
c_CH.SetBottomMargin(0.12)
c_CH.SetTopMargin(0.08)
CH_reco.Draw("COLZ")

c_CH.Draw()
c_CH.SaveAs(Out_dir + "nu_E_Reco_Distribution_CH.pdf")

f_out.cd()
CH_reco.GetValue().Write()

In [ ]:

ROOT.gStyle.SetOptStat(0)

nbin = 12

C_reco = df_flag.Filter('volume_flag == 1 && reco_flag == 1 && track_flag == 1 && space_flag == 1 && time_flag == 1 && Where_int == "Graphite" && target == "tgt:1000060120"').Histo1D(("C_reco", "C_reco", nbin, 0, 6000),"nu_Ereco")


c_C = TCanvas("c", "Canvas", 600, 500)
c_C.SetLeftMargin(0.20)
c_C.SetRightMargin(0.15)   # serve spazio per la barra dei colori
c_C.SetBottomMargin(0.12)
c_C.SetTopMargin(0.08)

C_reco.Draw("COLZ")

c_C.Draw()
c_C.SaveAs(Out_dir + "nu_E_Reco_Distribution_C.pdf")

f_out.cd()
C_reco.GetValue().Write()

Acceptance and efficiency 

In [ ]:
nbin= 12

h_num = df_flag.Filter('true_flag == 1 && volume_flag == 1 && reco_flag == 1 && track_flag == 1 && space_flag == 1 && time_flag == 1 && target == "tgt:1000010010"').Histo1D(("h_num", "h_num", nbin, 0, 6000),"nu_Ereco")

h_den_a = df_flag.Filter('volume_flag == 1 && reco_flag == 1 && track_flag == 1 && space_flag == 1 && time_flag == 1 && target == "tgt:1000010010"').Histo1D(("h_den_a", "h_den_a", nbin, 0, 6000),"nu_Ereco")
h_den_e = df_flag.Filter('true_flag == 1 && target == "tgt:1000010010"').Histo1D(("h_den_e", "h_den_e", nbin, 0, 6000),"nu_Ereco")



h_num_hist = h_num.GetValue()
h_den_hist_a = h_den_a.GetValue()
h_den_hist_e = h_den_e.GetValue()

h_accep = h_num_hist.Clone("h_accep")
h_accep.Divide(h_den_hist_a) 


h_eff = h_num_hist.Clone("h_eff")
h_eff.Divide(h_den_hist_e)

h_accep.SetTitle("Acceptance")
h_eff.SetTitle("Efficiency")
h_accep.SetName("h_accep")
h_eff.SetName("h_eff")

c = TCanvas("c", "Canvas", 600, 500)
c.Divide(2,1)
c.SetLeftMargin(0.12)
c.SetRightMargin(0.15)   
c.SetBottomMargin(0.12)
c.SetTopMargin(0.08)

c.cd(1)
h_accep.Draw()


c.cd(2)
h_eff.Draw()



c.Draw()
c.SaveAs(Out_dir + "Acceptance_Efficiency.pdf")



f_out.cd()
h_accep.Write()
h_eff.Write()


In [ ]:
f_out.Close()